In [346]:
import pandas as pd
import json
from datetime import datetime, date
from collections import defaultdict 

In [347]:

with open('re/DataSet.json', encoding='windows-1251') as f:
    data = json.load(f)


columns = ['client_1','client_2','client_3']



def create_df(name,json_data,columns):
    df_data = [data[name][i] for i in data[name]]
    df_index = dict.fromkeys([i for i in data[name]],[data[name][j] for j in data[name]]).keys()
    df = pd.DataFrame(data = df_data,index = df_index,columns=columns)
    return df


df_client = pd.DataFrame(data['client'], columns=columns)
df_data = create_df("dataLiability",data,columns)
df_source = create_df("source",data,columns)

df = df_client.append([df_data,df_source])

df


,client_1,client_2,client_3
name,Петров Николай Иванович,Иванов Петр Николаевич,Николаев Иван Петрович
clientType,ИП,ИП,ИП
birthday,09.07.2003,01.01.1990,11.11.1987
inn,123456789123,123456789124,123456789125
citizenship,RU,US,RU
appNO,"[V445B4, V45V24BB56B, F452V-454]","[SDF-S4B44-RTB, V24-VRQ-DV, 2-CQE-RV-EW3]","[24C-4-V4, V-2-4RTV-24-V-13V, CSDF-BW]"
dateStart,"[11.11.2016, 11.10.2017, 11.10.2018]","[11.11.2016, 11.10.2017, 11.10.2018]","[11.11.2016, 11.10.2017, 11.10.2018]"
dateEnd,"[, 01.01.2020, ]","[, , ]","[01.01.2020, 01.01.2020, 01.01.2020]"
sumLiability,"[100000, 555000, 100500]","[1000000, 99000, 125000]","[1500000, 200000, 350000]"
sumBalance,"[10000, 0, 50000]","[950000, 15000, 10000]","[0, 0, 0]"


### 2) Получить список уникальных обязательств (договоров) по каждому Заемщику

In [348]:
dict_uniq_contract = {}
for i in range(3):
    list_app = df.loc['appNO'][i]
    print('Cписок уникальных обязательств',{df.loc['name'][i]:list(set(list_app))})
    dict_uniq_contract[df.columns[i]] = list(set(list_app))
dict_uniq_contract

Cписок уникальных обязательств {'Петров Николай Иванович': ['V445B4', 'F452V-454', 'V45V24BB56B']}
Cписок уникальных обязательств {'Иванов Петр Николаевич': ['2-CQE-RV-EW3', 'V24-VRQ-DV', 'SDF-S4B44-RTB']}
Cписок уникальных обязательств {'Николаев Иван Петрович': ['24C-4-V4', 'V-2-4RTV-24-V-13V', 'CSDF-BW']}


{'client_1': ['V445B4', 'F452V-454', 'V45V24BB56B'],
 'client_2': ['2-CQE-RV-EW3', 'V24-VRQ-DV', 'SDF-S4B44-RTB'],
 'client_3': ['24C-4-V4', 'V-2-4RTV-24-V-13V', 'CSDF-BW']}

### 3) Посчитать количество открытых обязательств по каждому Заемщику. Обязательство считается открытым, если отсутствует дата закрытия.

In [349]:
dict_count_open_contact = {}
for i in range(len(df.columns)):
    filter_start = [x for x in df.loc['dateStart'][i] if x != '']
#     filter_start = filter(None,df.loc['dateStart'][i])
    filter_end = [x for x in df.loc['dateEnd'][i] if x != '']
    print('Количество открытых обязательств по заемщику {}'.format(df.loc['name'][i]), len(filter_start)-len(filter_end))
    dict_count_open_contact[df.columns[i]]= len(filter_start)-len(filter_end)
dict_count_open_contact

Количество открытых обязательств по заемщику Петров Николай Иванович 2
Количество открытых обязательств по заемщику Иванов Петр Николаевич 3
Количество открытых обязательств по заемщику Николаев Иван Петрович 0


{'client_1': 2, 'client_2': 3, 'client_3': 0}

### 4) Посчитать количество закрытых обязательств по каждому Заемщику. Если указана дата закрытия и она позже даты открытия.

In [350]:
dict_count_locked_contract = dict()
for i in range(len(df.columns)):
    filter_start = [datetime.strptime(d,'%d.%m.%Y').date() for d in df.loc['dateStart'][i] if d != '']
    filter_end = [datetime.strptime(d,'%d.%m.%Y').date() for d in df.loc['dateEnd'][i] if d != '']
    check_end = []
    for z in range(len(filter_end)):
        check_end = [k for k in filter_end if filter_end[z] > filter_start[z]] 
    print('количество закрытых обязательств по Заемщику {}'.format(df.loc['name'][i]), len(check_end))
    dict_count_locked_contract[df.columns[i]] = len(check_end)
print(dict_count_locked_contract)


количество закрытых обязательств по Заемщику Петров Николай Иванович 1
количество закрытых обязательств по Заемщику Иванов Петр Николаевич 0
количество закрытых обязательств по Заемщику Николаев Иван Петрович 3
{'client_1': 1, 'client_2': 0, 'client_3': 3}


### 5) Посчитать текущую просроченную задолженность по каждому Заемщику.

In [351]:
dict_sumDelay = dict()
for i in range(len(df.columns)):
    dict_sumDelay[df.columns[i]] = sum(df.loc['sumDelay'][i])
dict_sumDelay

{'client_1': 2500, 'client_2': 26000, 'client_3': 0}

### 6) Оценить кредитную историю каждого Заемщика: 

In [352]:
dict_statusCH = {}
#- КИ плохая, если текущая просрочка > 1000 руб.
#- КИ хорошая, если имеются закрытые/открытые договора и нет просрочки
#- КИ средняя, если имеется закрытые договора и 0 < текущая просрочка <= 1000 руб.
#- КИ отсутствует, если у клиента нет ни одного обязательства

# статус кредитной истории: G - хорошая, B - плохая, E - отсутствует, N - средняя

for i in df.columns:
    if dict_sumDelay[i] > 1000:
        dict_statusCH[i] = 'B'
    elif dict_count_locked_contract[i] != 0 and dict_sumDelay[i] == 0:
        dict_statusCH[i] = 'G'
    elif dict_count_locked_contract[i] != 0 and 0 < dict_sumDelay[i] <= 1000:
         dict_statusCH[i] = 'N'
    elif len(dict_uniq_contract[i]) == 0:
        dict_statusCH[i] = 'E'
dict_statusCH               

{'client_1': 'B', 'client_2': 'B', 'client_3': 'G'}

### 7) Реализовать правила, проверяющие на соответствие каждого клиента требованиям Банка и вернуть сработавшие правила:

In [353]:
#             - Гражданство РФ (в случае несоответствия вернуть правило "rule_1")
#             - Минимальный возраст - 18 лет (в случае несоответствия вернуть правило "rule_2")
#             - Максимальный возраст - 55 лет (в случае несоответствия вернуть правило "rule_3")
#             - ИП - статус "Действующий" (в случае несоответствия вернуть правило "rule_4")
#             - Срок ведения хоз. деятельности не менее 6-ти месяцев (в случае несоответствия вернуть правило "rule_5")

dict_rules = dict()

def count_years(birthday):
    now = datetime.now().date()
    last = datetime.strptime(birthday,'%d.%m.%Y').date()
    years = relativedelta(now, last).years
    return years


def count_month(birthday):
    now = datetime.now().date()
    last = datetime.strptime(birthday,'%d.%m.%Y').date()
    month = relativedelta(now, last).years * 12 + relativedelta(now, last).months
    return month



for i in range(len(df.columns)):
    dict_rules[df.columns[i]]=[]
    if df.loc['citizenship'][i] != 'RU':
        dict_rules[df.columns[i]].append('rule_1')
    if count_years(df.loc['birthday'][i]) < 18:
        dict_rules[df.columns[i]].append('rule_2')
    if count_years(df.loc['birthday'][i]) > 55:
        dict_rules[df.columns[i]].append('rule_3')
    if df.loc['egripStatus'][i] != 1:
        dict_rules[df.columns[i]].append('rule_4')
    if count_month(df.loc['dateRegistration'][i]) < 6:
        dict_rules[df.columns[i]].append('rule_5')
    

dict_rules
        

{'client_1': ['rule_2'],
 'client_2': ['rule_1'],
 'client_3': ['rule_4', 'rule_5']}

### 8) Вернуть ответ, пример ответа смотреть во вложении файл «Пример ответа». (файл ответа формировать не обязательно)

In [374]:
result = {'result':{'client':{}}}

for i in columns:
    client_dict = {str(i): {
				"rules": dict_rules[i],
				"countOpenLiability": dict_count_open_contact[i],
				"countCloseLiability": dict_count_locked_contract[i],
				"sumDelays": dict_sumDelay[i],
				"statusCH": dict_statusCH[i],
				"arrayLiability": dict_uniq_contract[i]
			},}
    result['result']['client'].update(client_dict)
 
result_json = json.dumps(result,indent=3)

#save in file
with open('result.json','w') as f:
    json.dump(result,f,indent = 3)

print(result_json)

{
   "result": {
      "client": {
         "client_1": {
            "rules": [
               "rule_2"
            ],
            "countOpenLiability": 2,
            "countCloseLiability": 1,
            "sumDelays": 2500,
            "statusCH": "B",
            "arrayLiability": [
               "V445B4",
               "F452V-454",
               "V45V24BB56B"
            ]
         },
         "client_2": {
            "rules": [
               "rule_1"
            ],
            "countOpenLiability": 3,
            "countCloseLiability": 0,
            "sumDelays": 26000,
            "statusCH": "B",
            "arrayLiability": [
               "2-CQE-RV-EW3",
               "V24-VRQ-DV",
               "SDF-S4B44-RTB"
            ]
         },
         "client_3": {
            "rules": [
               "rule_4",
               "rule_5"
            ],
            "countOpenLiability": 0,
            "countCloseLiability": 3,
            "sumDelays": 0,
            "statusCH"